# OpenAPI를 활용한 최저가 검색

* 네이버 개발자 센터
    * https://developers.naver.com/main/

## 로그인 정보

In [ ]:
import json
import urllib.request
import urllib.parse

In [ ]:
def load_config(filename='config.json'):
    with open(filename, 'r', encoding='euc-kr') as file:
        result = json.load(file)
    return result

In [ ]:
config = load_config(filename='my.json')
client_id, client_secret, display, sort_type = config['client_id'], config['client_secret'], config['display'], config['sort_type']

In [ ]:
client_id, client_secret, display, sort_type

## 검색 및 결과

In [ ]:
query = "샨링 M6"

* https://github.com/naver/naver-openapi-guide/blob/draft/sample/java/APIExamTranslateNMT.java

In [ ]:
def request_query(client_id, client_secret, query):
    url="https://openapi.naver.com/v1/search/shop.json"
    option="&display="+str(display)+"&start=1&sort="+sort_type
    url_query = url + "?query=" + urllib.parse.quote(query) + option
    
    request = urllib.request.Request(url_query)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if (rescode == 200):
        response_body = response.read()
        return response_body.decode('utf-8')
    else:
        return "Error code:"+response    

In [ ]:
result = request_query(client_id, client_secret, query)

## 검색어 정리

In [ ]:
def parse_response(response):
    json_data = json.loads(response)
    json_items = json_data["items"]
    result = []
    for item in json_items:        
        result.append({"title": item["title"], "link": item["link"], "lprice": item["lprice"], "mall_name": item["mallName"]})
    return result

In [ ]:
parse_response(result)[0:3]

In [ ]:
data = parse_response(result)

## Pandas를 활용한 데이터 활용

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data)
df

## DB에 저장하기

In [ ]:
import sqlite3
con = sqlite3.connect("naver_api.db")

In [ ]:
df.to_sql("price",con)

## DB에서 불러오기

In [ ]:
df_sql = pd.read_sql("select * from price", con)
df_sql